In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import re

In [2]:
def parse_query(text):
    pattern = r"<([^>]+)>(.*?)</\1>"
    matches = re.findall(pattern, text, re.DOTALL)
    return {key: value.strip() for key, value in matches}

In [6]:
file_name = 'compare_csv/arc_ai2_chall_train_top_50.csv'
df = pd.read_csv(file_name)
text_df = pd.read_csv(file_name[:-4] + '-text.csv')
display(df.head(3))
display(text_df.head(1))

,original,claude_v1,claude_v2,claude_v3,claude_v4,claude_v5,claude_v6,gemini_v1,gemini_pro_v1,gemini_v2,gemini_pro_v2,gemini_v2_think,gemini_v3,gemini_v3_think,gemini_pro_v1_think,gemini_v1_think,claude_v6_system_prompt,gemini_v1_few_options,claude_v1_few_options,claude_v1_refine
0,<question>Which land form is the result of the...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו תצורת נוף היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו צורת נוף היא תוצאה של כוח בונה ...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו צורת נוף היא תוצאה של כוח בונה ...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו תצורת נוף היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזה צורת נוף היא תוצאה של כוח בונה ...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...
1,<question>Which statement best compares single...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר או...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזו טענה משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר או...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...
2,"<question>As part of an experiment, an astrona...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח מאזניים ...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח מאזניים ...","<question>כחלק מניסוי, אסטרונאוט לוקח מאזניים ...","<question>כחלק מניסוי, אסטרונאוט לוקח מאזניים ...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>במסגרת ניסוי, אסטרונאוט לוקח מאזניים...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר..."


,original,claude_v1_few_options text,claude_v1_refine text
0,<question>Which land form is the result of the...,I'll translate the question and answer options...,First translation:\n<question>איזו צורת קרקע ה...


In [7]:
def color_gradient(val):
    val = val / 50
    # g = int(255 * val ** 2)
    g = int(255 * val ** 0.5)
    r = 255 - g
    return f'background-color: rgb({r}, {g}, 0)'

In [8]:
corr = {}
for c_1 in df.columns:
    for c_2 in df.columns:
        corr[(c_1, c_2)] = (df[c_1] == df[c_2]).sum()

use_keys = list(df.columns)
use_keys = sorted(use_keys)

for sub in ['claude', 'gemini']:
    print(sub)
    use_keys_sub = [k for k in use_keys if sub in k]
    corr_df = pd.DataFrame(index=use_keys_sub, columns=use_keys_sub)
    for k_1 in use_keys_sub:
        for k_2 in use_keys_sub:
            corr_df.loc[k_1, k_2] = corr[(k_1, k_2)]
    styled_df = corr_df.style.format('{:.3f}').map(color_gradient)
    display(styled_df)
    print()

claude


,claude_v1,claude_v1_few_options,claude_v1_refine,claude_v2,claude_v3,claude_v4,claude_v5,claude_v6,claude_v6_system_prompt
claude_v1,50.000,14.000,6.000,26.000,22.000,23.000,21.000,24.000,26.000
claude_v1_few_options,14.000,50.000,4.000,14.000,14.000,15.000,14.000,15.000,14.000
claude_v1_refine,6.000,4.000,50.000,4.000,4.000,4.000,4.000,5.000,4.000
claude_v2,26.000,14.000,4.000,50.000,31.000,28.000,26.000,31.000,23.000
claude_v3,22.000,14.000,4.000,31.000,50.000,29.000,29.000,30.000,25.000
claude_v4,23.000,15.000,4.000,28.000,29.000,50.000,30.000,35.000,24.000
claude_v5,21.000,14.000,4.000,26.000,29.000,30.000,50.000,30.000,23.000
claude_v6,24.000,15.000,5.000,31.000,30.000,35.000,30.000,50.000,22.000
claude_v6_system_prompt,26.000,14.000,4.000,23.000,25.000,24.000,23.000,22.000,50.000



gemini


,gemini_pro_v1,gemini_pro_v1_think,gemini_pro_v2,gemini_v1,gemini_v1_few_options,gemini_v1_think,gemini_v2,gemini_v2_think,gemini_v3,gemini_v3_think
gemini_pro_v1,50.000,16.000,7.000,6.000,3.000,3.000,1.000,0.000,0.000,4.000
gemini_pro_v1_think,16.000,50.000,7.000,7.000,6.000,5.000,3.000,1.000,1.000,5.000
gemini_pro_v2,7.000,7.000,50.000,3.000,2.000,1.000,3.000,2.000,2.000,3.000
gemini_v1,6.000,7.000,3.000,50.000,10.000,19.000,12.000,10.000,6.000,12.000
gemini_v1_few_options,3.000,6.000,2.000,10.000,50.000,8.000,5.000,6.000,5.000,10.000
gemini_v1_think,3.000,5.000,1.000,19.000,8.000,50.000,8.000,9.000,9.000,10.000
gemini_v2,1.000,3.000,3.000,12.000,5.000,8.000,50.000,13.000,12.000,10.000
gemini_v2_think,0.000,1.000,2.000,10.000,6.000,9.000,13.000,50.000,14.000,13.000
gemini_v3,0.000,1.000,2.000,6.000,5.000,9.000,12.000,14.000,50.000,10.000
gemini_v3_think,4.000,5.000,3.000,12.000,10.000,10.000,10.000,13.000,10.000,50.000


In [46]:
from nltk.translate.meteor_score import meteor_score as meteor_score_func
import nltk

nltk.download('wordnet')

def calc_meteor_score(references, candidates):
    scores = []
    for i in range(len(references)):
        scores.append(meteor_score_func(references[i], candidates[i]))

    return np.mean(scores)

[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
calc_meteor_score(
    [[i.split()] for i in df['claude_v1'].values],
    [i.split() for i in df['gemini_pro_v1'].values],
)

0.8097958112385422

In [47]:
calc_meteor_score(
    [[i.split()] for i in df['gemini_pro_v1'].values],
    [i.split() for i in df['claude_v1'].values],
)

0.8199473899271658

In [40]:
print(df['original'].iloc[31])

<question>Lisa, Jack, and Tommy earned $60 from washing cars all week. However, half of the $60 was earned by Lisa. Tommy earned half of what Lisa earned. How much more money did Lisa earn than Tommy?</question>
<answer>Lisa earned $60 * 1/2 = $<<60*1/2=30>>30.
Tommy earned $30 * 1/2 = $<<30*1/2=15>>15.
Lisa earned $30 - $15 = $<<30-15=15>>15 more than Tommy.
#### 15</answer>


In [26]:
def parse_answer(text):
    return int(text.split('####')[1].split('<')[0])

In [31]:
for c in df.columns:
    if (df['original'].apply(parse_answer) != df[c].apply(parse_answer)).sum() > 0:
        print(c)

# Compare Claude few-shots

In [9]:
[k for k in use_keys if 'claude' in k]

['claude_v1',
 'claude_v1_few_options',
 'claude_v1_refine',
 'claude_v2',
 'claude_v3',
 'claude_v4',
 'claude_v5',
 'claude_v6',
 'claude_v6_system_prompt']

In [10]:
[k for k in use_keys if 'gemini' in k]

['gemini_pro_v1',
 'gemini_pro_v1_think',
 'gemini_pro_v2',
 'gemini_v1',
 'gemini_v1_few_options',
 'gemini_v1_think',
 'gemini_v2',
 'gemini_v2_think',
 'gemini_v3',
 'gemini_v3_think']

In [11]:
# old = 'claude_v4'

one_v = 'claude_v1_refine'
two_v = 'gemini_pro_v1_think'

for i in range(50):
    row = df.iloc[i]
    if row[one_v] == row[two_v]:
        continue
    else:
        print(i, '\n')
        # old_parsed = parse_query(row[old])
        one_parsed = parse_query(row[one_v])
        two_parsed = parse_query(row[two_v])
        for key in one_parsed:
            if one_parsed[key] != two_parsed[key]:
                print(f'{key}:\n\t{one_parsed[key]}\n\t{two_parsed[key]}\n')
                # print(f'{key}:\n\t{one_parsed[key]}\n\t{two_parsed[key]}\n{old}:\n{old_parsed[key]}\n')
            #     continue
            # else:
            #     print(f'{key}:\n\t{one_parsed[key]}\n\t{two_parsed[key]}\n')
        print('\n---------------------------\n')

0 

question:
	איזו צורת נוף היא תוצאה של הכוח הבונה של קרחון?
	איזו תצורת נוף היא תוצאה של הכוח הבונה של קרחון?

option 2:
	ערימות סלעים שהושקעו על ידי קרחון נמס
	ערמות סלעים שהושקעו על ידי קרחון נמס

option 4:
	גבעות סלע יסוד שהתחספסו ממעבר קרחון
	גבעות סלע אם שחוספסו על ידי מעבר של קרחון


---------------------------

1 

question:
	איזה היגד משווה בצורה הטובה ביותר בין אורגניזמים חד-תאיים לרב-תאיים?
	איזה היגד משווה בצורה הטובה ביותר בין יצורים חד-תאיים לרב-תאיים?

option 1:
	הרקמות באורגניזם חד-תאי דומות לתאים באורגניזם רב-תאי.
	רקמות ביצור חד-תאי הן כמו התאים ביצור רב-תאי.

option 2:
	הגרעין באורגניזם חד-תאי דומה לעור של אורגניזם רב-תאי.
	גרעין התא ביצור חד-תאי הוא כמו העור של יצור רב-תאי.

option 3:
	האברונים באורגניזם חד-תאי דומים לאיברים באורגניזם רב-תאי.
	אברונים ביצור חד-תאי הם כמו האיברים ביצור רב-תאי.

option 4:
	הציטופלזמה באורגניזם חד-תאי דומה למערכת העצבים באורגניזם רב-תאי.
	הציטופלזמה ביצור חד-תאי היא כמו מערכת העצבים ביצור רב-תאי.


---------------------------

2 

qu

In [ ]:
for i in range(50):
    print(i)
    print(text_df.loc[i, 'claude_v1_refine text'])
    print('\n- - - - - - - - - - - - - - - - - - - - - - - - - - \n')

# Compare Gemini few-shots

In [ ]:
print(df['original'].iloc[20])

In [ ]:
[k for k in use_keys if 'gemini' in k]

In [38]:
# old = 'claude_v4'

one_v = 'original'
two_v = 'gemini_pro_v1'

for i in range(50):
    row = df.iloc[i]
    if row[one_v] == row[two_v]:
        continue
    else:
        print(i, '\n')
        # old_parsed = parse_query(row[old])
        one_parsed = parse_query(row[one_v])
        two_parsed = parse_query(row[two_v])
        for key in one_parsed:
            if one_parsed[key] != two_parsed[key]:
                print(f'{key}:\n\t{one_parsed[key]}\n\t{two_parsed[key]}\n')
                # print(f'{key}:\n\t{one_parsed[key]}\n\t{two_parsed[key]}\n{old}:\n{old_parsed[key]}\n')
        print('\n---------------------------\n')

0 

question:
	Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?
	למארק יש גינה עם פרחים. הוא שתל בה צמחים בשלושה צבעים שונים. עשרה מהם צהובים, ויש 80% יותר פרחים סגולים. יש רק 25% פרחים ירוקים ממספר הפרחים הצהובים והסגולים יחד. כמה פרחים יש למארק בגינה שלו?

answer:
	There are 80/100 * 10 = <<80/100*10=8>>8 more purple flowers than yellow flowers.
So in Mark's garden, there are 10 + 8 = <<10+8=18>>18 purple flowers.
Purple and yellow flowers sum up to 10 + 18 = <<10+18=28>>28 flowers.
That means in Mark's garden there are 25/100 * 28 = <<25/100*28=7>>7 green flowers.
So in total Mark has 28 + 7 = <<28+7=35>>35 plants in his garden.
#### 35
	יש 80/100 * 10 = <<80/100*10=8>>8 יותר פרחים סגולים מאשר פרחים צהובים.
אז בגינה של מארק יש 10 + 8 = <<10+8=18>>18 פרחים סג

In [39]:
for i in range(50):
    print(i)
    print(text_df.loc[i, 'gemini_pro_v1 text'])
    print('\n- - - - - - - - - - - - - - - - - - - - - - - - - - \n')

0
**Thought Process Summary:**

Okay, here's what I've been thinking. The task is to translate an English word problem into Hebrew, and then do the same for the solution. I started by breaking down the English question sentence by sentence, considering different phrasing options in Hebrew to find the most natural and clear way to express each part. I made sure to retain the core meaning of the problem, while adjusting for grammatical nuances and idiomatic expressions in Hebrew.

I went through several iterations, refining the Hebrew translation of the question. I paid particular attention to the sentence structure and word choice, ensuring it was appropriate for a grade-school level understanding, to make it easier for the learner. I finally landed on a version that felt both accurate and natural in Hebrew.

Next, I tackled the solution. I translated each step of the English solution into Hebrew, making sure the mathematical expressions and calculations were correctly rendered. I kept 